In [1]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1707, done.
remote: Counting objects: 100% (1707/1707), done.
remote: Compressing objects: 100% (943/943), done.
remote: Total 1707 (delta 803), reused 1620 (delta 757), pack-reused 0
Receiving objects: 100% (1707/1707), 81.14 MiB | 12.16 MiB/s, done.
Resolving deltas: 100% (803/803), done.
Checking out files: 100% (1935/1935), done.


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [3]:
# !python3 /content/idl-spring-22-project-deepar/src/data/company_data.py
!python3 /content/idl-spring-22-project-deepar/src/data/company_data_two_head.py

100% 5/5 [00:07<00:00,  1.53s/it]
{0: 'MCD', 1: 'BEEM', 2: 'XXII', 3: 'RWB', 4: 'PSTV'}
{'MCD': 0, 'BEEM': 1, 'XXII': 2, 'RWB': 3, 'PSTV': 4}
train labels:  (10908, 192, 2)
train inputs:  (10908, 192, 5)
100% 5/5 [00:03<00:00,  1.60it/s]
test labels:  (4236, 192, 2)
test inputs:  (4236, 192, 5)
total files:  10
discarded files:  0
Well formed files:  10


In [4]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [5]:
print(index_to_company)
print(company_to_index)

{0: 'MCD', 1: 'BEEM', 2: 'XXII', 3: 'RWB', 4: 'PSTV'}
{'MCD': 0, 'BEEM': 1, 'XXII': 2, 'RWB': 3, 'PSTV': 4}


In [6]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [7]:
NUM_TRAIN_SAMPLES = 10000
NUM_TEST_SAMPLES  = 3800

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [8]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (10000, 192, 5)
stock_labels_trimmed shape (10000, 192, 2)
stock_test_inputs_trimmed shape (3800, 192, 5)
stock_test_labels_trimmed shape (3800, 192, 2)


In [9]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])




In [10]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))


[0.0, 1.0, 2.0, 3.0, 4.0]
5
[0.0, 1.0, 2.0, 4.0]
4


In [11]:

params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':5,
    'cov_dim': 4,
    'lstm_hidden_dim': 64,
    'lstm_layers':4 ,
    'window_size':192,
    'batch_size': 96,
    'learning_rate': 1e-3,
    'epochs':20,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24
}

print(params)

{'num_classes': 5, 'embedding_dim': 5, 'cov_dim': 4, 'lstm_hidden_dim': 64, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 96, 'learning_rate': 0.001, 'epochs': 20, 'num_test_samples': 3800, 'num_train_samples': 10000, 'conditioning_period': 168, 'prediction_period': 24}


In [12]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index].astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2  

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index].astype(np.float32))
      x2 = torch.from_numpy(self.label[index].astype(np.float32))

      return x1, x2


In [13]:
def unwindow_sequence(dataset):
  company_data = {}

  seq_id = 0
  company_index_cov_id = -1

  for cov_ip, cov_op in dataset:

    comp_index = cov_ip[seq_id][company_index_cov_id]
    company_name = index_to_company[comp_index.item()]

    # IP Shape (Time, Dim)
    # OP Shape (Time, 1) # One covariate (open price) per time instant
    if company_data.get(company_name, None) is None:
      company_data[company_name] = [cov_ip, cov_op]
      
    else:
      # windows are created with stride 1
      # so we should be appending only the last element in the sequence
      cov_ip_last = cov_ip[-1, :].unsqueeze(0)
      cov_op_last = cov_op[-1].unsqueeze(0)

      # 0 --> Covariates
      # 1 --> Outputs, aka stock price
      company_data[company_name][0] = torch.cat((company_data[company_name][0], cov_ip_last), dim=0)
      company_data[company_name][1] = torch.cat((company_data[company_name][1], cov_op_last), dim=0)

  return company_data

In [14]:
train_data = TrainDataset()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

train_data_unwindowed = unwindow_sequence(train_data)
test_data_unwindowed = unwindow_sequence(test_data)

In [15]:
print(train_data_unwindowed.keys())
print(len(train_data_unwindowed))

dict_keys(['MCD', 'BEEM', 'XXII', 'RWB', 'PSTV'])
5


In [16]:
print(test_data_unwindowed.keys())
print(len(test_data_unwindowed))
# Some Companies are there in the Train Set, But not in the Test Set. 
# However, all companies in the Test Set is there in the Train Set

dict_keys(['MCD', 'BEEM', 'XXII', 'PSTV'])
4


In [17]:
import plotly.express as px

def plot_output(company_data, company_name,covariate_index=0):
  # 1st index is output (prediction)
  x=np.linspace(1, company_data[company_name][1].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("op", company_data[company_name][0][:,covariate_index])
  

def plot_input_covariate(company_data, company_name, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[company_name][0].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("ip", company_data[company_name][0][:,covariate_index])

In [18]:
# SANITY Check One: Output Label (open price) is shifted by one element of input (the ip opening price)
plot_output(train_data_unwindowed, 'MCD',0)
plot_input_covariate(train_data_unwindowed, 'MCD', 0)

op tensor([-0.7642, -0.7593, -0.7721,  ...,  0.1128,  0.0853,  0.0925])


ip tensor([-0.7642, -0.7593, -0.7721,  ...,  0.1128,  0.0853,  0.0925])


In [19]:

# SANITY Check TWO: Train and Test data are contiguous
plot_input_covariate(train_data_unwindowed, 'MCD', 2)
plot_input_covariate(test_data_unwindowed, 'MCD', 2)


ip tensor([-1.6811, -1.5670, -1.4528,  ...,  1.5150, -1.6811, -1.5670])


ip tensor([-1.2245, -0.8821, -0.7679,  ..., -1.6811, -1.5670, -1.4528])


In [20]:
# SANITY Check TWO for Output Labels: Train and Test data are contiguous

plot_output(train_data_unwindowed, 'MCD')
plot_output(test_data_unwindowed, 'MCD')

op tensor([-0.7642, -0.7593, -0.7721,  ...,  0.1128,  0.0853,  0.0925])


op tensor([0.0867, 0.0878, 0.0803,  ..., 2.6597, 2.6452, 2.6950])


In [21]:
plot_input_covariate(train_data_unwindowed, 'MCD', 0)  # Opening Price
plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Volume
plot_input_covariate(train_data_unwindowed, 'MCD', 2)  # Day
plot_input_covariate(train_data_unwindowed, 'MCD', 3)  # Month
plot_input_covariate(train_data_unwindowed, 'MCD', 4)  # Company ID


ip tensor([-0.7642, -0.7593, -0.7721,  ...,  0.1128,  0.0853,  0.0925])


ip tensor([-0.5096, -0.9862,  0.5291,  ...,  0.2002, -0.2303, -0.3687])


ip tensor([-1.6811, -1.5670, -1.4528,  ...,  1.5150, -1.6811, -1.5670])


ip tensor([ 1.2869,  1.2869,  1.2869,  ..., -0.4512, -0.1615, -0.1615])


ip tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [22]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id,covariate_index = 1):
  # 1st index is output (prediction)
  x=np.linspace(1, company_data[window_id][1].shape[0], num=len(company_data[window_id][1][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][1][:,covariate_index])
  fig.show()
  print("op", company_data[window_id][1][:,covariate_index])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])


In [23]:
plot_input_covariate_windowed(test_data, 0, 0)
plot_output_windowed(test_data, 0,0)
plot_output_windowed(test_data, 0,1)


ip tensor([0.0867, 0.0878, 0.0803, 0.0724, 0.0800, 0.0787, 0.0680, 0.0603, 0.0649,
        0.0830, 0.0940, 0.0927, 0.1128, 0.0996, 0.0988, 0.0899, 0.0976, 0.0867,
        0.0737, 0.0899, 0.0861, 0.0867, 0.0823, 0.0979, 0.1075, 0.1192, 0.1211,
        0.1216, 0.1367, 0.1145, 0.1158, 0.1130, 0.1147, 0.1334, 0.1100, 0.0878,
        0.0954, 0.1111, 0.1128, 0.1452, 0.1507, 0.1423, 0.1480, 0.1463, 0.1395,
        0.1426, 0.1301, 0.1235, 0.1203, 0.1357, 0.1349, 0.1507, 0.1577, 0.1621,
        0.1409, 0.0184, 0.0795, 0.0396, 0.0852, 0.0900, 0.0842, 0.0373, 0.0831,
        0.1004, 0.0729, 0.0927, 0.1081, 0.0715, 0.0795, 0.1092, 0.1081, 0.1295,
        0.1282, 0.1007, 0.1111, 0.1054, 0.1073, 0.1009, 0.1175, 0.1035, 0.1108,
        0.1225, 0.1271, 0.1087, 0.1549, 0.1704, 0.1624, 0.1797, 0.1978, 0.1927,
        0.1954, 0.2158, 0.1910, 0.2248, 0.2172, 0.2143, 0.2132, 0.3140, 0.3262,
        0.3427, 0.3434, 0.3343, 0.3495, 0.3529, 0.3463, 0.3379, 0.3421, 0.3485,
        0.3454, 0.3619, 0.3512, 0.365

op tensor([0.0878, 0.0803, 0.0724, 0.0800, 0.0787, 0.0680, 0.0603, 0.0649, 0.0830,
        0.0940, 0.0927, 0.1128, 0.0996, 0.0988, 0.0899, 0.0976, 0.0867, 0.0737,
        0.0899, 0.0861, 0.0867, 0.0823, 0.0979, 0.1075, 0.1192, 0.1211, 0.1216,
        0.1367, 0.1145, 0.1158, 0.1130, 0.1147, 0.1334, 0.1100, 0.0878, 0.0954,
        0.1111, 0.1128, 0.1452, 0.1507, 0.1423, 0.1480, 0.1463, 0.1395, 0.1426,
        0.1301, 0.1235, 0.1203, 0.1357, 0.1349, 0.1507, 0.1577, 0.1621, 0.1409,
        0.0184, 0.0795, 0.0396, 0.0852, 0.0900, 0.0842, 0.0373, 0.0831, 0.1004,
        0.0729, 0.0927, 0.1081, 0.0715, 0.0795, 0.1092, 0.1081, 0.1295, 0.1282,
        0.1007, 0.1111, 0.1054, 0.1073, 0.1009, 0.1175, 0.1035, 0.1108, 0.1225,
        0.1271, 0.1087, 0.1549, 0.1704, 0.1624, 0.1797, 0.1978, 0.1927, 0.1954,
        0.2158, 0.1910, 0.2248, 0.2172, 0.2143, 0.2132, 0.3140, 0.3262, 0.3427,
        0.3434, 0.3343, 0.3495, 0.3529, 0.3463, 0.3379, 0.3421, 0.3485, 0.3454,
        0.3619, 0.3512, 0.3654, 0.362

op tensor([-3.2214e-01,  5.7738e-01, -2.5589e-01, -4.3281e-01, -5.7484e-01,
         3.6707e-02, -7.4691e-01, -5.0726e-01, -1.7874e-01,  4.6525e-01,
        -5.2078e-01, -3.3715e-01, -5.1119e-01, -4.0568e-01,  8.7046e-02,
         7.5915e-01,  2.6164e-01, -3.6419e-01, -7.0270e-01, -5.3233e-01,
        -2.6558e-01, -2.3162e-01, -7.0710e-02, -1.5659e-01, -3.2667e-01,
         3.9283e-03,  5.2120e-01,  5.5622e-01,  1.9757e-01, -2.2234e-01,
        -4.3210e-01, -9.6331e-02,  1.3165e+00, -5.0359e-02,  2.4485e-02,
        -2.4615e-01,  7.7708e-02, -1.0194e-01,  5.4229e-01, -3.6956e-01,
        -8.1848e-01, -6.3076e-01,  2.8240e-01, -6.4244e-01, -4.9210e-01,
        -6.0808e-01, -4.3186e-01, -3.1035e-01, -7.2120e-01, -1.4956e-01,
        -4.1770e-01,  7.2646e-02,  2.6505e-01,  1.4861e+00,  2.6447e+00,
         1.6245e+00,  3.0265e+00,  1.2616e+00, -3.5721e-02, -1.8312e-01,
         4.4635e-01,  3.3298e-01,  1.6475e-01, -3.6903e-01,  1.0736e-02,
        -1.7071e-01,  1.3556e-01,  2.6137e-01, -

In [24]:
print('train data shape', train_data[0][0].shape)

train data shape torch.Size([192, 5])


# MODEL


In [25]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.price_mu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.price_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.price_sigma = nn.Softplus()

        self.volume_mu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.volume_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.volume_sigma = nn.Softplus()



    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        onehot_embed = self.embedding(company_index)
        
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_price_mu = self.price_mu(out1)
        out_price_presigma = self.price_presigma(out1)
        out_price_sigma = self.price_sigma(out_price_presigma)

        out_volume_mu = self.volume_mu(out1)
        out_volume_presigma = self.volume_presigma(out1)
        out_volume_sigma = self.volume_sigma(out_volume_presigma)

        return out_price_mu, out_price_sigma, out_volume_mu, out_volume_sigma, hn_cn



In [26]:
model = Network(params)

In [27]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_price_mu, out_price_sigma, out_volume_mu, out_volume_sigma, ht_ct = model.forward(x=ip_covariate)

  print('price mu shape', out_price_mu.shape)
  print('price sigma shape', out_price_sigma.shape)

  print('volume mu shape', out_volume_mu.shape)
  print('volume sigma shape', out_volume_sigma.shape)

  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

price mu shape torch.Size([96, 192, 1])
price sigma shape torch.Size([96, 192, 1])
volume mu shape torch.Size([96, 192, 1])
volume sigma shape torch.Size([96, 192, 1])
ht shape torch.Size([4, 96, 64]) torch.Size([4, 96, 64])


In [28]:
def loss_fn(mu,sigma,labels):
  mu = torch.squeeze(mu)
  sigma = torch.squeeze(sigma)
  total_likelihood = 0
  for i in range(mu.shape[0]):  # loop through batch
    for j in range(mu.shape[1]):# each batch  loop through time steps
      distribution = torch.distributions.normal.Normal(mu[i][j] , sigma[i][j]) # scaling the values by 10 to avoid small sigma values
      total_likelihood += distribution.log_prob(labels[i][j])
  return -1*total_likelihood/(mu.shape[0]*mu.shape[1])

In [29]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  mu = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  diff = np.sum((mu - labels) ** 2, axis=1)
  summation = np.sum(np.abs(labels), axis=1)
  result = (np.sqrt(diff) / summation)
  n = len(result)
  result = np.sum(result)/n
  return result

In [30]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  diff = np.sum((predictions - labels) ** 2, axis=1)
  summation = np.sum(np.abs(labels), axis=1)
  result = (np.sqrt(diff) / summation)
  n = len(result)
  result = np.sum(result)/n
  return result

In [31]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [38]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_price_mu = None
  overall_volume_mu = None
  overall_price_sigma = None
  overall_volume_sigma = None
  overall_price_label = None
  overall_volume_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      # print("ip_covariate: ",ip_covariate.shape)
      # print("op_label: ",op_label.shape)
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period,:]
      # print("cond_ip: ",cond_ip.shape)
      # print("cond_op: ",cond_op.shape)
      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period:,: ]
      # print("pred_ip: ",pred_ip.shape)
      # print("pred_op: ",pred_op.shape) 
      # Step One - Forward Pass : Conditioning Period    
      price_mu, price_sigma, volume_mu, volume_sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_price_mu = price_mu.squeeze()
      batch_price_sigma = price_sigma.squeeze()
      batch_volume_mu = volume_mu.squeeze()
      batch_volume_sigma = volume_sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_price_mu, pred_price_sigma = price_mu[:, -1, :].unsqueeze(1), price_sigma[:, -1, :].unsqueeze(1)
      pred_volume_mu, pred_volume_sigma = volume_mu[:, -1, :].unsqueeze(1), volume_sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze( 1)
        pred_cov_ip[:, 0, 0] = pred_price_mu[:, 0, 0]
        pred_cov_ip[:,0,1] = pred_volume_mu[:, 0, 0]
        
        price_pred_mu, price_pred_sigma, volume_pred_mu, volume_pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_price_mu = torch.cat((batch_price_mu,  price_pred_mu.squeeze(2)), dim=1)
        batch_price_sigma = torch.cat((batch_price_sigma, price_pred_sigma.squeeze(2)), dim=1)

        batch_volume_mu = torch.cat((batch_volume_mu,  volume_pred_mu.squeeze(2)), dim=1)
        batch_volume_sigma = torch.cat((batch_volume_sigma, volume_pred_sigma.squeeze(2)), dim=1)


      if overall_price_mu is None and overall_price_sigma is None and overall_volume_mu is None and overall_volume_sigma is None:
        overall_price_mu = batch_price_mu
        overall_price_sigma = batch_price_sigma
        overall_volume_mu = batch_volume_mu
        overall_volume_sigma = batch_volume_sigma
        overall_price_label = op_label[:,  :,0]
        overall_volume_label = op_label[:,  :,1]
      else:
        overall_price_mu = torch.cat((overall_price_mu,  batch_price_mu), dim=0)
        overall_volume_mu = torch.cat((overall_volume_mu,  batch_volume_mu), dim=0)
        
        overall_price_sigma = torch.cat((overall_price_sigma, batch_price_sigma), dim=0)
        overall_volume_sigma = torch.cat((overall_volume_sigma, batch_price_sigma), dim=0)
        
        overall_price_label = torch.cat((overall_price_label, op_label[:,  :,0]), dim=0)
        overall_volume_label = torch.cat((overall_volume_label, op_label[:, :,1]), dim=0)

  rmse = accuracy_RMSE_test(overall_price_mu, overall_price_label)
  
  rmse += accuracy_RMSE_test(overall_volume_mu, overall_volume_label)

  return rmse







In [39]:
validate(model)

price
torch.Size([3800, 192])
torch.Size([3800, 192])
torch.Size([3800, 192])
torch.Size([3800, 192])
volume


0.19793901945415296

In [ ]:
# rmse = accuracy_RMSE_train(overall_mu, overall_label)
# print("rmse ", rmse)

In [ ]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])


# TRAIN

In [48]:
model_version='deepar_model_refined_v1.pt'
epochs = params['epochs']

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0

    if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
        # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

        checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch'] + 1

    for i, (ip_covariate, op_label) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        op_label     = op_label.cuda()

        price_mu, price_sigma, volume_mu, volume_sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        loss = loss_fn(price_mu, price_sigma, op_label[:,:,0])
        loss += loss_fn(volume_mu,volume_sigma,op_label[:,:,1])

        # print("loss", loss)

        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse= validate(model=model)
    scheduler.step()

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
      torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'val_rmse': val_rmse,
              'loss': total_loss/len(train_loader),
              },  f'/content/drive/MyDrive/DeepARExperiments/best_dev_acc_{best_dev_rmse}_{model_version}')
    
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse)

Network(
  (embedding): Embedding(5, 5)
  (lstm): LSTM(9, 64, num_layers=4, batch_first=True)
  (price_mu): Linear(in_features=64, out_features=1, bias=True)
  (price_presigma): Linear(in_features=64, out_features=1, bias=True)
  (price_sigma): Softplus(beta=1, threshold=20)
  (volume_mu): Linear(in_features=64, out_features=1, bias=True)
  (volume_presigma): Linear(in_features=64, out_features=1, bias=True)
  (volume_sigma): Softplus(beta=1, threshold=20)
)


price
torch.Size([3800, 192])
torch.Size([3800, 192])
torch.Size([3800, 192])
torch.Size([3800, 192])
volume


FileNotFoundError: ignored

In [ ]:
checkpoint = torch.load(f'/content/deepar_model_refined_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [44]:


model.eval()
model.cuda()


total_RMSE = 0

price_overall_mu = None
price_overall_sigma = None
price_overall_label = None

volume_overall_mu = None
volume_overall_sigma = None
volume_overall_label = None

with torch.no_grad():
  for i, (ip_covariate, op_label) in enumerate(test_loader):
    ip_covariate = ip_covariate.cuda()
    op_label     = op_label.cuda()

    cond_ip = ip_covariate[:, 0:conditionining_period, :]
    cond_op = op_label[:, 0:conditionining_period]

    pred_ip = ip_covariate[:, conditionining_period: , :]
    pred_op = op_label[:,  conditionining_period:,: ]

    # Step One - Forward Pass : Conditioning Period    
    price_mu, price_sigma,volume_mu,volume_sigma, ht_ct = model(x=cond_ip, h0_c0=None)

    price_batch_mu = price_mu.squeeze()
    price_batch_sigma = price_sigma.squeeze()
    
    volume_batch_mu = volume_mu.squeeze()
    volume_batch_sigma = volume_sigma.squeeze()

    # Initialize pred_mu for the first time instance of the "prediction period"
    # from the value of the "predicted mu" from the last instance of the "conditioning period"
    price_pred_mu, price_pred_sigma = price_mu[:, -1, :].unsqueeze(1), price_sigma[:, -1, :].unsqueeze(1)
    volume_pred_mu, volume_pred_sigma = volume_mu[:, -1, :].unsqueeze(1), volume_sigma[:, -1, :].unsqueeze(1)

    for t in range(0, prediction_period):
      pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
      pred_cov_ip[:, 0, 0] = price_pred_mu[:, 0, 0]
      pred_cov_ip[:, 0, 1] = volume_pred_mu[:, 0, 0]

      price_pred_mu, price_pred_sigma,volume_pred_mu, volume_pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
      
      price_batch_mu = torch.cat((price_batch_mu,  price_pred_mu.squeeze(2)), dim=1)
      price_batch_sigma = torch.cat((price_batch_sigma, price_pred_sigma.squeeze(2)), dim=1)

      volume_batch_mu = torch.cat((volume_batch_mu,  volume_pred_mu.squeeze(2)), dim=1)
      volume_batch_sigma = torch.cat((volume_batch_sigma, volume_pred_sigma.squeeze(2)), dim=1)


    if price_overall_mu is None and price_overall_sigma is None:
      price_overall_mu = price_batch_mu
      price_overall_sigma = price_batch_sigma
      price_overall_label = op_label[:,:,0]

      volume_overall_mu = volume_batch_mu
      volume_overall_sigma = volume_batch_sigma
      volume_overall_label = op_label[:,:,1]
    else:
      price_overall_mu = torch.cat((price_overall_mu,  price_batch_mu), dim=0)
      price_overall_sigma = torch.cat((price_overall_sigma, price_batch_sigma), dim=0)
      price_overall_label = torch.cat((price_overall_label, op_label[:,:,0]), dim=0)

      volume_overall_mu = torch.cat((volume_overall_mu,  volume_batch_mu), dim=0)
      volume_overall_sigma = torch.cat((volume_overall_sigma, volume_batch_sigma), dim=0)
      volume_overall_label = torch.cat((volume_overall_label, op_label[:,:,1]), dim=0)

In [46]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, window_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))
  fig.show()

In [47]:
# #s for the paper 3500 420
plot_test_output_data(price_overall_label.cpu().numpy(),price_overall_mu.cpu().numpy(), 2500  )
plot_test_output_data(volume_overall_label.cpu().numpy(),volume_overall_mu.cpu().numpy(), 2500  )